In [ ]:
import torch
from torch.utils import data
import faiss
import numpy as np
import os
import json
from tqdm import tqdm
from datetime import datetime

from dataset_utils import ImageNet, Places365, ArtPlaces

from evaluation_utils import create_model, extract_features, calculate_accuracy, calculate_precision, calculate_recall

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print(torch.cuda.get_device_name())

### Konstanten

In [ ]:
from evaluation_constants import DEST, DISTANCE, K, MODELS, DATASETS

### Datensätze

In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = 8

imagenet = ImageNet(root=r"C:\Users\mariu\Documents\Studium\Praktikum\ImageNet_Evaluation_Subset")
imagenet_dataloader = data.DataLoader(imagenet, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

places365 = Places365(root=r"C:\Users\mariu\Documents\Studium\Praktikum\Places365_Evaluation_Subset")
places365_dataloader = data.DataLoader(places365, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

artplaces = ArtPlaces(root=r"C:\Users\mariu\Documents\Development\Datasets\ArtPlaces_13371280")
artplaces_dataloader = data.DataLoader(artplaces, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

# viridis, plasma, rainbow
datasets = {
    "ImageNet": imagenet,
    "ImageNet_Loader": imagenet_dataloader,
    "Places365": places365,
    "Places365_Loader": places365_dataloader,
    "ArtPlaces": artplaces,
    "ArtPlaces_Loader": artplaces_dataloader,
}

### Features extrahieren und evaluieren

In [ ]:
# for m in MODELS:
#     model = create_model(m, device)

#     for d in DATASETS:
#         dataloader = datasets[d + "_Loader"]
#         _, (b, l) = next(enumerate(dataloader))
#         b = b.to(device)

#         print(f"Extracting features ({m["name"]}, {d}): {model(b).shape}")

In [ ]:
results = {}

for m in MODELS:
    results[m["name"]] = {}
    model = create_model(m)

    for d in DATASETS:
        results[m["name"]][d] = {}

        dataloader = datasets[d + "_Loader"]

        tqdm_dataloader = tqdm(dataloader, unit="batch")
        tqdm_dataloader.set_description(f"Extracting features ({m["name"]}, {d})")

        features, labels = extract_features(model, tqdm_dataloader)
        features = np.array(features).astype("float32")
        labels = np.array(labels)

        match DISTANCE:
            case "L2":
                index = faiss.IndexFlatL2(features.shape[1])
            case "cosine":
                faiss.normalize_L2(features)
                index = faiss.IndexFlatIP(features.shape[1])
        
        index.add(features)

        _, I = index.search(features, max(K) + 1)  # + 1 to account for self-match
        I = I[:, 1:]  # Remove self-match

        for k in K:
            results[m["name"]][d]["Accuracy@" + str(k)] = calculate_accuracy(I, labels, k)
            results[m["name"]][d]["Precision@" + str(k)] = calculate_precision(I, labels, k)
            results[m["name"]][d]["Recall@" + str(k)] = calculate_recall(I, labels, k)

In [ ]:
dest = os.path.join(DEST, datetime.today().strftime('%Y%m%d_%H%M%S'))

os.makedirs(dest)

with open(os.path.join(dest, "results.json"), mode="w+") as file:
    json.dump(results, file)

with open(os.path.join(dest, "DISTANCE.json"), mode="w+") as file:
    json.dump(DISTANCE, file)

with open(os.path.join(dest, "K.json"), mode="w+") as file:
    json.dump(K, file)

with open(os.path.join(dest, "DATASETS.json"), mode="w+") as file:
    json.dump(DATASETS, file)

with open(os.path.join(dest, "MODELS.json"), mode="w+") as file:
    json.dump(MODELS, file)